In [ ]:
# Import all necessary packages to run functions
from pathlib import Path
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from FlowCytometryTools import FCMeasurement
from FlowCytometryTools import QuadGate, ThresholdGate
import sklearn
from sklearn import preprocessing
from sklearn.decomposition import PCA
import seaborn as sns
from seaborn import relplot


def appPCA(data, features):
    """Applies the PCA algorithm to the data set"""
    # Apply PCA to the data set
    # setting values of data of selected features to data frame
    xi = data.loc[:, features].values
    # STANDARDIZE DATA --> very important to do before applying machine learning algorithm
    scaler = preprocessing.StandardScaler()
    xs = scaler.fit_transform(xi)
    #xs = sklearn.preprocessing.scale(xi)
    xs = np.nan_to_num(xs)
    # setting how many components wanted --> PC1 and PC2
    pca = PCA(n_components=2)
    # apply PCA to standardized data set
    # NOTE: score == xf
    xf = pca.fit(xs).transform(xs)
    # creates the loading array (equation is defintion of loading)
    loading = pca.components_.T
    return xf, loading
    
    
    
def loadingPlot(loading, features, i, title):
    """Plot the loading data"""
    # Loading
    # Create graph for loading values
    x_load = loading[:, 0]
    y_load = loading[:, 1]

    # Create figure for the loading plot
    fig1 = plt.figure(figsize=(8, 8))
    ax = fig1.add_subplot(1, 1, 1)
    ax.set_xlabel("PC1", fontsize=15)
    ax.set_ylabel("PC2", fontsize=15)
    plt.scatter(x_load, y_load)

    for z, feature in enumerate(features):
        # Please note: not the best logic, but there are three features in NK and four features in T cells
        if len(features) == 4:
            name = "T Cells"
            if feature == "BL1-H":
                feature = "Foxp3"
            elif feature == "VL1-H":
                feature = "CD25"
            elif feature == "VL4-H":
                feature = "CD4"
            elif feature == "BL3-H":
                feature = "CD45RA"
        if len(features) == 3:
            name = "NK Cells"
            if feature == "VL4-H":
                feature = "CD3"
            if feature == "RL1-H":
                feature = "CD8"
            if feature == "BL1-H":
                feature = "CD56"
        plt.annotate(str(feature), xy=(x_load[z], y_load[z]))
    ax.set_title(name + " - Loading - " + str(title), fontsize=20)
    
    
def pcaAll(sampleType, check, titles):
    """
    Use to plot the score and loading graphs for PCA. Assign protein and pstat5 arrays AND score and loading arrays
    This is all the data for each file.
    Want to use for both T and NK cells? Use it twice!
    sampleType is importF for T or NK
    check == "t" for T cells OR check == "n" for NK cells
    """
    # declare the arrays to store the data
    data_array = []
    pstat_array = []
    xf_array = []
    loading_array = []
    
    # create the for loop to file through the data and save to the arrays
    # using the functions created above for a singular file
    if check == "t":
        for i, sample in enumerate(sampleType):
            title = titles[i].split("/")
            title = title[len(title)-1]
            data, pstat, features = sampleT(sample)
            data_array.append(data)
            pstat_array.append(pstat)
            xf, loading = appPCA(data, features)
            xf_array.append(xf)
            loading_array.append(loading)
            pcaPlt(xf, pstat, features, i, title)
            loadingPlot(loading, features, i, title)
            #plt.scatter(data)
    elif check == "n":
        for i, sample in enumerate(sampleType):
            title = titles[i].split("/")
            title = title[len(title)-1]
            data, pstat, features = sampleNK(sample)
            data_array.append(data)
            pstat_array.append(pstat)
            xf, loading = appPCA(data, features)
            pcaPlt(xf, pstat, features, i, title)
            loadingPlot(loading, features, i, title)
    plt.show()
    return data_array, pstat_array, xf_array, loading_array

def count_data(sampleType, gate):
    """
    Used to count the number of cells and store the data of all of these cells in a folder with multiple files --> automates the process sampleType
    is NK or T cell data, gate is the desired cell population.
    Sample type: is the overall importF assignment for T or NK (all the T cell files, all NK cell files)
    """
    # declare the arrays to store the data
    count_array = []
    data_array = []
    # create the for loop to file through the data and save to the arrays
    # using the functions created above for a singular file
    for _, sample in enumerate(sampleType):
        #samplel = sample.transform("hlog", channels=["BL1-H", "VL1-H", "VL4-H", "RL1-H"]) #new
        count_array.append(cellCount(sample, gate))
        data_array.append(rawData(sample, gate))
    # returns the array for count of cells and the array where each entry is the data for the specific cell population in that .fcs file
    return count_array, data_array

def cellCount(sample_i, gate):
    """
    Function for returning the count of cells in a single .fcs. file of a single cell file. Arguments: single sample/.fcs file and the gate of the
    desired cell output.
    """
    # Import single file and save data to a variable --> transform to logarithmic scale
    smpl = sample_i.transform("hlog", channels=["BL1-H", "VL1-H", "VL4-H", "RL1-H"])
    # Apply T reg gate to overall data --> i.e. step that detrmines which cells are T reg
    cells = smpl.gate(gate)
    # Number of events (AKA number of cells)
    cell_count = cells.get_data().shape[0]
    # print('Number of Treg cells:' + str(treg_count))
    return cell_count

def rawData(sample_i, gate):
    """
    Function that returns the raw data of certain cell population in a given file. Arguments: sample_i is a single entry/.fcs file and the gate
    of the desired cell population.
    """
    smpl = sample_i.transform("hlog", channels=["BL1-H", "VL1-H", "VL4-H", "RL1-H"])
    # Apply T reg gate to overall data --> i.e. step that detrmines which cells are T reg
    cells = smpl.gate(gate)
    # Get raw data of t reg cells in file
    cell_data = cells.get_data()
    return cell_data

def protscatterT(data,channel):
    channels = ["BL1-H", "VL1-H", "VL4-H", "BL3-H"]
    proteins = ["Foxp3", "CD25", "CD4", "CD45RA"]
    pstat = data[["RL1-H"]]
    fig1 = plt.figure(figsize=(8, 8))
    ax = fig1.add_subplot(1, 1, 1)
    xlabel = proteins[channels == channel]
    ax.set_xlabel(xlabel, fontsize=15)
    ax.set_ylabel("Pstat Activity", fontsize=15)
    plt.scatter(data[channel], pstat, s = 1.2, c = "mediumaquamarine", alpha = 1)
    ax.set_title("Activity to Protein Relation",fontsize=20)

def pcaAllTypeSpec(sampleType, check, titles):
    """
    Use to plot the score and loading graphs for PCA. Assign protein and pstat5 arrays AND score and loading arrays
    This is all the data for each file.
    Want to use for both T and NK cells? Use it twice!
    sampleType is importF for T or NK
    check == "t" for T cells OR check == "n" for NK cells
    """
    # declare the arrays to store the data
    data_array = []
    pstat_array = []
    xf_array = []
    loading_array = []
    
    # create the for loop to file through the data and save to the arrays
    # using the functions created above for a singular file
    if check == "t":
        celltypeT = [treg, nonTreg]
        celltypeTS = ["treg", "nonTreg"]
        for i, cell in enumerate (celltypeT): 
            gate = celltypeT[i]()
            _, alldata = count_data(sampleType,gate)
            for j, sample in enumerate(sampleType):
                tform = alldata[j]#.transform("hlog", channels=["BL1-H", "VL1-H", "VL4-H", "BL3-H", "RL1-H"])
                data = tform[["BL1-H", "VL1-H", "VL4-H", "BL3-H"]]
                pstat = tform[["RL1-H"]]
                title = titles[j].split("/")
                title = title[len(title)-1] + " " + celltypeTS[i]
                _, _, features = sampleT(sample)
                if len(alldata[j]) != 0:
                    xf, loading = appPCA(alldata[j], features)
                    xf_array.append(xf)
                    loading_array.append(loading)
                    pcaPlt(xf, pstat, features, j, title)
                    loadingPlot(loading, features, j, title)
                protscatterT(tform,"BL1-H")
                plt.show()
                
    elif check == "n":
        celltypeN = [nk, bnk, cd]
        celltypeNS = ["nk", "bnk", "cd"]
        for i, cell in enumerate (celltypeN):
            gate = celltypeN[i]()
            _, alldata = count_data(sampleType,gate)
            for j, sample in enumerate(sampleType):
                tform = alldata[j]#.transform("hlog", channels=["VL4-H", "RL1-H", "BL1-H", "BL2-H"])
                data = tform[["VL4-H", "RL1-H", "BL1-H"]]
                pstat = tform[["BL2-H"]]
                title = titles[j].split("/")
                title = title[len(title)-1] + " " + celltypeNS[i]
                _, _, features = sampleNK(sample)
                if len(alldata[j]) != 0:
                    xf, loading = appPCA(alldata[j], features)
                    pcaPlt(xf, pstat, features, j, title)
                    loadingPlot(loading, features, j, title)
                plt.show()
    #plt.show()
    return  xf_array, loading_array



In [ ]:
tplate = input('What is the name of the T plate folder?:')
tsample, titlesT = importF(tplate)

nkplate = input('What is the name of the NK plate folder?:')
nksample, titlesNK = importF(nkplate)

xfT_array, loadingT_array = pcaAllTypeSpec(tsample, "t", titlesT)

xfNK_array, loadingNK_array = pcaAllTypeSpec(nksample, "n", titlesNK)
#/home/brianoj/PosterPlatesT
